In [1]:
import pandas as pd
import re, requests, getpass
import os, glob, time, subprocess
import ipywidgets as widgets
from bs4 import BeautifulSoup

In [2]:
os.chdir('/mnt/benzinger3/norah/py_scripts/status_helper')
# Populate our data
# Add the pet_status information for each respective csv file
pet_statuses = pd.read_csv('pet_statuses.csv')
mr_statuses = pd.read_csv('mr_statuses.csv')

In [3]:
# Change working directory to where most recent reports are located
os.chdir('/mnt/benzinger3/dian_proc_report/data')
# Set the current time to the date of the most recent report
now = time.strftime("%Y%m%d")
# //////////////////////////
# /////  MANUTAL TIME  ////
# ////////////////////////
# ls -l -rt | tail -n 4
now = '20231218'
# Read the most recent report information from the respecte mr and pup files
dian_mrfs_data = pd.read_csv(f"{now}_dian_mrfs_data.csv")
dian_petpup_data = pd.read_csv(f"{now}_dian_petpup_data.csv")
# Filter the data so it only includes what we will need for our status_set_scripts
filtered_dian_mrfs_data = dian_mrfs_data[['session','accession','mr_status']]
filtered_dian_petpup_data = dian_petpup_data[['session','accession','pet_status']]

In [4]:
# Generate MR selection widget
selected_mr_status = []
# Load Functions: 
def display_mr_status_buttons():
    # Create button for each mr_status
    buttons = []
    for button in [i for i in mr_statuses.iloc[:, 0]]:
        if pd.isna(button):
            button = "null"
        button = widgets.Button(description=str(button))
        buttons.append(button)
    # Create callback function for button click
    def on_button_click(b):
        selected_mr_status.clear()  # clear previously selected buttons
        selected_mr_status.append(b.description)
        # Update selection widget
        mr_selection_widget.value = f"Current selection: {b.description}"
    # Attach callback function to buttons
    for button in buttons:
        button.on_click(on_button_click)
    # Define the layout for the buttons
    grid_layout = widgets.Layout(
        grid_template_columns='repeat(5, 150px)',
        grid_gap='10px'
    )
    # Create a grid of buttons
    button_grid = widgets.GridBox(buttons, layout=grid_layout)
    # Display the grid of buttons
    display(button_grid)
    
# Generate PUP selection widget
selected_pet_status = []
# Load Functions: 
def display_pet_status_buttons():
    # Create button for each mr_status
    buttons = []
    for button in [i for i in pet_statuses.iloc[:, 0]]:
        button = widgets.Button(description=button)
        buttons.append(button)
    # Create callback function for button click
    def on_button_click(b):
        selected_pet_status.clear()  # clear previously selected buttons
        selected_pet_status.append(b.description)
        # Update selection widget
        pup_selection_widget.value = f"Current selection: {b.description}"
    # Attach callback function to buttons
    for button in buttons:
        button.on_click(on_button_click)
    # Define the layout for the buttons
    grid_layout = widgets.Layout(
        grid_template_columns='repeat(5, 150px)',
        grid_gap='10px'
    )
    # Create a grid of buttons
    button_grid = widgets.GridBox(buttons, layout=grid_layout)
    # Display the grid of buttons
    display(button_grid)
    
    
# Generate data collection widget
def get_accessions():
    global paired_data
    formatted_inputted_data = inputted_data.split('\n')
    # create an empty list to hold the session-accession pairs
    session_accession_pairs = []
    # iterate through each row of the filtered_dian_petpup_data data frame
    for index, row in filtered_dian_petpup_data.iterrows(): #  and filtered_dian_mrfs_data.iterrows():
        # check if the session value in the row is in the input list
        if row['session'] in formatted_inputted_data:
            # if it is, add a tuple of the session and accession values to the pairs list
            session_accession_pairs.append((row['session'], row['accession']))
    # iterate through each row of the filtered_dian_mrfs_data data frame
    for index, row in filtered_dian_mrfs_data.iterrows():
        # check if the session value in the row is in the input list
        if row['session'] in formatted_inputted_data:
            # if it is, add a tuple of the session and accession values to the pairs list
            session_accession_pairs.append((row['session'], row['accession']))          
    # create a new data frame from the session-accession pairs list
    paired_data = pd.DataFrame(session_accession_pairs, columns=['session', 'accession'])
    # display the new data frame
    print(paired_data)
    
def get_sessions():
    print("Paste your data here: ")
    text_box = widgets.Textarea(
        value='',
        placeholder="""for example:
session_1
session_2
session_3""",
        description='',
        disabled=False
    )

    submit_button = widgets.Button(description='Submit')
    output = widgets.Output()
    # On click button
    def on_submit_button_clicked(b):
        with output:
            global inputted_data
            inputted_data = text_box.value
            print('Data saved and accessions identified:')
            # Call helper function
            get_accessions()

    submit_button.on_click(on_submit_button_clicked)
    # Generate display
    display(text_box)
    display(submit_button)
    display(output)

In [5]:
# Create MR selection widget
mr_selection_widget = widgets.HTML(value="Current MR selection: ")
print("Select an MR status option: ")
display_mr_status_buttons()
# Display selection widget
display(mr_selection_widget)

Select an MR status option: 


GridBox(children=(Button(description='Usable', style=ButtonStyle()), Button(description='1.5T Scanner', style=…

HTML(value='Current MR selection: ')

In [6]:
# Create PET selection widget
pup_selection_widget = widgets.HTML(value="Current PUP selection: ")
print("Select an PET status option: ")
display_pet_status_buttons()
# Display selection widget
display(pup_selection_widget)

Select an PET status option: 


GridBox(children=(Button(description='Usable', style=ButtonStyle()), Button(description='MRI Unavailable', sty…

HTML(value='Current PUP selection: ')

In [7]:
get_sessions()

Paste your data here: 


Textarea(value='', placeholder='for example:\nsession_1\nsession_2\nsession_3')

Button(description='Submit', style=ButtonStyle())

Output()

In [9]:
paired_data

,session,accession
0,3011006_MR_GOLE_US17,DCA74_E00172
1,3011006_MR_GOLE_US18,DCA75_E00915
2,3011006_MR_GOLE_US19,DCA76_E00247
3,3031005_MR_GOLE_US10,DCA70_E00048
4,4011002_MR_GOLE_TOSM1020,DCA75_E00324
5,7011006_MR_GOLE_TOSM1500,DCA63_E01385
6,7011015_MR_GOLE_OSM1,DIANTU46_E01775
7,7011016_MR_GOLE_OSM1,DIANTU46_E00659
8,1044003_MR_E2814_V8,DCA91_E00053
9,2044004_MR_E2814_V2,DCA88_E00241


In [10]:
# MR
accession_list = [i for i in paired_data.iloc[:, 1]]
########-- ACCESSIONS --##########
print("Selected accessions: ")
print(accession_list)

#########-- MR STATUS --##########
MRStatus = str(selected_mr_status[0])
print(f"Selected MR Status: {MRStatus}")

print("MR STATUS CHANGES:")
for i in accession_list:
    pet_accession_number = i
    print(i + ' -> ' + MRStatus)
    # Updates MR status

Selected accessions: 
['DCA74_E00172', 'DCA75_E00915', 'DCA76_E00247', 'DCA70_E00048', 'DCA75_E00324', 'DCA63_E01385', 'DIANTU46_E01775', 'DIANTU46_E00659', 'DCA91_E00053', 'DCA88_E00241', 'DCA91_E00020']
Selected MR Status: Usable
MR STATUS CHANGES:
DCA74_E00172 -> Usable
DCA75_E00915 -> Usable
DCA76_E00247 -> Usable
DCA70_E00048 -> Usable
DCA75_E00324 -> Usable
DCA63_E01385 -> Usable
DIANTU46_E01775 -> Usable
DIANTU46_E00659 -> Usable
DCA91_E00053 -> Usable
DCA88_E00241 -> Usable
DCA91_E00020 -> Usable


In [ ]:
# PET
accession_list = [i for i in paired_data.iloc[:, 1]]
########-- ACCESSIONS --##########
print("Selected accessions: ")
print(accession_list)

#########-- PET STATUS --#########
PETStatus = str(selected_pet_status[0])
print(f"Selected PET Status: {PETStatus}")

print("PET STATUS CHANGES:")
for i in accession_list:
    pet_accession_number = i
    print(i + ' -> ' + PETStatus)
    # Updates PET status

In [11]:
def login():
    # Set the URL of the jsession webpage
    jsession_url = 'https://dca.wustl.edu/data/JSESSION'
    # Get password, response, and soup object
    password = getpass.getpass("Enter XNAT password: ")
    response = requests.get(jsession_url, auth=('nvii', password))
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    jsession_token = str(soup)
    # Send a request to the webpage and get the HTML response
    response = requests.get(jsession_url)
    html = response.text
    # Create a Beautiful Soup object from the HTML
    soup = BeautifulSoup(html, "html.parser")
    # Convert the soup into a usable token 
    # Wipe the password from memory
    password = None; del password
    return jsession_token

jsession_token = login()

Enter XNAT password:  ········


In [12]:
os.chdir('/mnt/benzinger3/norah/py_scripts/status_helper')
print(os.getcwd())

/mnt/benzinger3/norah/py_scripts/status_helper


In [13]:
note = 'Manual ROIs'
note = 'null'

In [ ]:
# Updates PET status
for i in accession_list:
    accession_number = i
    subprocess.run(['./set_dca_petstatus_jsession.sh', jsession_token, accession_number, PETStatus, note], capture_output=True).stdout.decode('utf-8')
    print(f"{i} updated to {PETStatus}.")

In [14]:
# Updates MR status
for i in accession_list:
    accession_number = i
    subprocess.run(['./set_dca_mrstatus_jsession.sh', jsession_token, accession_number, MRStatus, note], capture_output=True).stdout.decode('utf-8')
    print(f"{i} updated to {MRStatus}.")

DCA74_E00172 updated to Usable.
DCA75_E00915 updated to Usable.
DCA76_E00247 updated to Usable.
DCA70_E00048 updated to Usable.
DCA75_E00324 updated to Usable.
DCA63_E01385 updated to Usable.
DIANTU46_E01775 updated to Usable.
DIANTU46_E00659 updated to Usable.
DCA91_E00053 updated to Usable.
DCA88_E00241 updated to Usable.
DCA91_E00020 updated to Usable.
